# Figure S5: Comparison with Rounce et al. data

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd
    
import os
from glob import glob
from oggm import utils
from tqdm import tqdm
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

os.chdir("/home/rooda/Dropbox/Patagonia/")

In [ ]:
RGI6_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")
RGI7_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI7_v2.shp")
RGI6_ids = RGI6_ids[RGI6_ids.area_km2 > 1][["RGIId", "Zone"]]
RGI7_ids = RGI7_ids[RGI7_ids.area_km2 > 1]

# RGI6 doesnt have IDs yet 
RGI7_ids = utils.cook_rgidf(RGI7_ids, o1_region='17', o2_region='02', bgndate= RGI7_ids.src_date, 
                            version = "70", assign_column_values= {'Zone' : 'Zone'})

RGI7_ids = RGI7_ids[["RGIId", "Zone"]]
ids = pd.concat([RGI6_ids, RGI7_ids]).set_index("RGIId")

dict_zone = {1:'PPY', 2:'PCA', 3:'NPI-E', 4:'NPI-W', 5:'SPI-N', 6:'SPI-C', 7:'SPI-S', 8:'GCN', 9:'CDI'}
ids = ids.replace({"Zone": dict_zone})

## Rounce et al. data

In [ ]:
df = xr.concat([xr.open_dataset("MS2 Results/rounce_et_al/R17_glac_mass_annual_50sets_2000_2100-ssp126.nc"),
               xr.open_dataset("MS2 Results/rounce_et_al/R17_glac_mass_annual_50sets_2000_2100-ssp245.nc"),
               xr.open_dataset("MS2 Results/rounce_et_al/R17_glac_mass_annual_50sets_2000_2100-ssp370.nc"),
               xr.open_dataset("MS2 Results/rounce_et_al/R17_glac_mass_annual_50sets_2000_2100-ssp585.nc")], dim = "SSP")
df = df.glac_mass_annual.drop_vars(["lon", "lat"])
df.coords["SSP"] = ["ssp126", "ssp245", "ssp370", "ssp585"]
df.coords["model"] = df.Climate_Model
df.coords["glacier"] = df.RGIId
df = df.drop_vars(["RGIId", "Climate_Model"])
df = df.where(df.glacier.isin(ids.index), drop = True)

ids_subset = ids[ids.index.isin(df.glacier.to_pandas().tolist())]
df = df.assign_coords(glacier = ids_subset.Zone.tolist())
df = df.groupby('glacier').sum()

df = 1 - (df.sel(year = 2100) / df.sel(year = 2015))
df_mean_rounce = df.mean(dim = "model").to_pandas().transpose()
df_var_rounce = df.std(dim = "model").to_pandas().transpose()

## Aguayo et al. data

In [ ]:
def preprocess(ds): # remove unnecessary variables and coordinates
    return ds.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])['volume']

In [ ]:
# Folder with all the results
gdirs = glob("/home/rooda/OGGM_results/new/*", recursive = True)

ds    = []

for gdir in tqdm(gdirs):

    # read historical run 
    model_hist   = xr.open_mfdataset(gdir + "/run_outputs_*.nc", preprocess = preprocess)
    model_hist   = model_hist.sel(time=2015).volume # check NAs

    paths = glob(gdir + "/run_output_*ssp*.nc", recursive = True)
    for path in tqdm(paths, leave = False):

        # read future run and concatenate
        model_future = xr.open_dataset(path)
        model_future = preprocess(model_future).sel(time=2100)
        model   = xr.concat([model_hist, model_future], dim = "time").load()

        # add basin ID to each glacier ID (RGI_ID)
        ids_subset = ids[ids.index.isin(model.rgi_id.to_pandas().tolist())]
        model = model.assign_coords(rgi_id = ids_subset.Zone.tolist())
        model = model.groupby('rgi_id').sum()
        model = 1 - (model.sel(time = 2100) / model.sel(time = 2015))
        
        # ID of the setup
        experiment_id = pd.Series(data = {'SSP':     os.path.basename(path).split("_")[3]})
        ds_model = pd.DataFrame(pd.concat([experiment_id, model.to_pandas()]), columns=['mass_loss']).transpose()
        ds.append(ds_model)
        
ds = pd.concat(ds)
df_mean_aguayo = ds.groupby("SSP").mean().transpose()
df_var_aguayo = ds.groupby("SSP").std().transpose()

In [ ]:
# colors
cl = px.colors.colorbrewer.RdYlBu
symbols = ['circle', 'diamond','square', 'cross', 'hexagram','pentagon', 'hexagon', 'arrow', "x"]

fig = go.Figure(data=go.Scatter(x=df_mean_rounce.ssp126, y=df_mean_aguayo.ssp126, 
                                error_y=dict(type='data', array=df_var_aguayo.ssp126, thickness=0.2, width=3),
                                error_x=dict(type='data', array=df_var_rounce.ssp126, thickness=0.2, width=3),
                                mode='markers', name = "SSP 1-2.6", marker=dict(size=15, color = cl[9], opacity=0.8, symbol = symbols)))
fig.add_trace(go.Scatter(x=df_mean_rounce.ssp245, y=df_mean_aguayo.ssp245, 
                         error_y=dict(type='data', array=df_var_aguayo.ssp245, thickness=0.2, width=3),
                         error_x=dict(type='data', array=df_var_rounce.ssp245, thickness=0.2, width=3),
                         mode='markers',name = "SSP 2-4.5", marker=dict(size=15, color = cl[8], opacity=0.8, symbol = symbols)))

fig.add_trace(go.Scatter(x=df_mean_rounce.ssp370, y=df_mean_aguayo.ssp370, 
                        error_y=dict(type='data', array=df_var_aguayo.ssp370, thickness=0.2, width=3),
                        error_x=dict(type='data', array=df_var_rounce.ssp370, thickness=0.2, width=3),
                         mode='markers', name = "SSP 3-7.0", marker=dict(size=15, color = cl[3], opacity=0.8, symbol = symbols)))

fig.add_trace(go.Scatter(x=df_mean_rounce.ssp585, y=df_mean_aguayo.ssp585, 
                         error_y=dict(type='data', array=df_var_aguayo.ssp585, thickness=0.2, width=3),
                         error_x=dict(type='data', array=df_var_rounce.ssp585, thickness=0.2, width=3),                         
                         mode='markers', name = "SSP 5-8.5", marker=dict(size=15, color = cl[1], opacity=0.8, symbol = symbols)))

fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name = "1:1", line=dict(color="rgba(0,0,0,0.3)", dash= 'dot')))

fig.update_yaxes(title = "Volume loss in 2100 (rel. to 2015) [Rounce et al. 2023]", range = [0.4, 1], tickformat = ',.0%',
                 ticks="outside", zeroline=False, griddash = "dot", gridcolor = "rgba(0,0,0,0.1)", showline = True, linecolor = 'black', linewidth = 0.2, mirror=True)

fig.update_xaxes(title = "Volume loss in 2100 (rel. to 2015) [this study]", range = [0.4, 1], tickformat = ',.0%',
                 ticks="outside", zeroline=False, griddash = "dot", gridcolor = "rgba(0,0,0,0.1)", showline = True, linecolor = 'black', linewidth = 0.2, mirror=True)

fig.add_annotation(text="CDI", font=dict(size=14), ax=30, ay=30, x=0.68, y=0.565, showarrow=True, arrowhead=0, arrowwidth=1, arrowcolor="black")
fig.add_annotation(text="GCN", font=dict(size=14), ax=30, ay=30, x=0.85, y=0.79, showarrow=True, arrowhead=0, arrowwidth=1, arrowcolor="black")
fig.add_annotation(text="NPI-E", font=dict(size=14), ax=40, ay=40, x=0.59, y=0.52, showarrow=True, arrowhead=0, arrowwidth=1, arrowcolor="black")
fig.add_annotation(text="SPI-S", font=dict(size=14), ax=50, ay=50, x=0.765, y=0.69, showarrow=True, arrowhead=0, arrowwidth=1, arrowcolor="black")
fig.add_annotation(text="PCA", font=dict(size=14), ax=-30, ay=-30, x=0.79, y=0.84, showarrow=True, arrowhead=0, arrowwidth=1, arrowcolor="black")
fig.add_annotation(text="PPY", font=dict(size=14), ax=-40, ay=-40, x=0.868, y=0.865, showarrow=True, arrowhead=0, arrowwidth=1, arrowcolor="black")
fig.add_annotation(text="SPI-C", font=dict(size=14), ax=-40, ay=-40, x=0.667, y=0.67, showarrow=True, arrowhead=0, arrowwidth=1, arrowcolor="black")
fig.add_annotation(text="SPI-N", font=dict(size=14), ax=-40, ay=-40, x=0.515, y=0.535, showarrow=True, arrowhead=0, arrowwidth=1, arrowcolor="black")
fig.add_annotation(text="NPI-W", font=dict(size=14), ax=-10, ay=-60, x=0.53, y=0.56, showarrow=True, arrowhead=0, arrowwidth=1, arrowcolor="black")

fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)")
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)")    
   

fig.update_layout(legend=dict(y=0.99, x=0.01, bgcolor = 'rgba(0,0,0,0.0)'))
fig.update_layout(height=600, width=600, template = "seaborn", margin = dict(l=20, r=20, b=30, t=30), hovermode = False)

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_S5_comparison_rounce.png", scale=4)
#fig.show()